In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import AnimalShelter class from CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
shelter = AnimalShelter()
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 30002
database = "AAC"
collection = "animals"
shelter.set_username(username)
shelter.set_password(password)
shelter.set_host(host)
shelter.set_port(port)
shelter.set_database(database)
shelter.set_collection(collection)

# Connect to database via CRUD Module
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True, errors='ignore')

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'grazios_logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Place the HTML image tag in the layout according to your design
app.layout = html.Div([
    html.Center(html.Img(src=f'data:image/png;base64, {encoded_image.decode()}', style={'height': '100px'})),
    html.Center(html.B(html.H1('Grazios Salvare Dashboard'))),
    html.Center(html.P("Erika Rodriguez", style={'font-size': 'small', 'font-style': 'italic'})),
    html.Hr(),
    
    html.Div([
        html.Label("Animal Filter"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ], style={'text-align': 'center'}),
    
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'height': '400px', 'overflowY': 'auto'},
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'color': 'white',
        },
        filter_action="native",
        sort_action="native",
        row_selectable="single",
    ),
    html.Br(),
    html.Hr(),
    
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever"]}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute"]}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Border Collie", "Australian Shepherd"]}}
    else: 
        query = {}
        
    filtered_df = pd.DataFrame.from_records(shelter.read(query))
    filtered_df.drop(columns=['_id'], inplace=True, errors='ignore')
    return filtered_df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(view_data):
    if view_data is None or len(view_data) == 0:
        return []
    
    dff = pd.DataFrame.from_dict(view_data)
    fig = px.pie(dff, names='breed', title='Breed Distribution of Filtered Animals')
    return [dcc.Graph(figure=fig)]

# Callback to highlight selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the map for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(view_data, index):  
    if view_data is None:
        return []
    elif index is None:
        return []
    
    dff = pd.DataFrame.from_dict(view_data)
    row = index[0] if index else 0  # Default to the first row if no row selected
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Successfully connected to MongoDB at nv-desktop-services.apporto.com:30002
Successfully connected to MongoDB at nv-desktop-services.apporto.com:30002
Successfully connected to MongoDB at nv-desktop-services.apporto.com:30002
Successfully connected to MongoDB at nv-desktop-services.apporto.com:30002
Successfully connected to MongoDB at nv-desktop-services.apporto.com:30002
Successfully connected to MongoDB at nv-desktop-services.apporto.com:30002
Successfully connected to MongoDB at nv-desktop-services.apporto.com:30002
['animals']
Dash app running on http://127.0.0.1:31423/
['animals']
